[Last.fm ](https://www.last.fm/login?next=/api/account/create)

* username: metaversegods
* email: fergut2000@gmail.com
* password: Elgaleron2000#
* API: 912b321d8a267e7892932b6deccb3d2c
* Shared secret: bf15bdeab91fe35c3ecbfd11f0d9458c

* example URL JSON: http://ws.audioscrobbler.com/2.0/?method=album.getinfo&api_key=51de025812af79cb70f4a872936181a0&artist=Cher&album=Believe&format=json



* [lastfm to csv](https://benjaminbenben.com/lastfm-to-csv/ )

Notes for how it would work:
* Users need to create a Last.fm account and connect to spotify 

https://github.com/pylast/pylast 

* [user.getTopArtists ](https://www.last.fm/api/show/user.getTopArtists)
* [get_top_artists(self, period='overall', limit=None)](https://lastfm-docs.github.io/api-docs/user/getTopArtists/)

* get_userplaycount(self)


In [ ]:
!pip install pylast
!pip install requests-toolbelt

import pylast

     |████████████████████████████████| 84 kB 2.3 MB/s 
     |████████████████████████████████| 68 kB 6.9 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
     |████████████████████████████████| 79 kB 5.2 MB/s 
     |████████████████████████████████| 54 kB 587 kB/s 


# Getting Scrobbles from [Github](https://github.com/mathieuhendey/lastfm_downloader)

* https://www.last.fm/user/metaversegods/library?from=2022-01-01&to=2022-04-20&page=1


In [ ]:
import sys
import pandas as pd
import requests
import pprint
from requests_toolbelt.threaded import pool

In [ ]:
# Generate your own at https://www.last.fm/api/account/create

TEXT = '#text'
ESTIMATED_TIME_FOR_PROCESSING_PAGE = 352
ESTIMATED_TIME_FOR_PROCESSING_DATAFRAME_PER_PAGE_OF_RESULTS = 275

In [ ]:
group_size = input("how many friends are in your group?: ")

for i in range(len(group_size)):
  LASTFM_API_KEY = input("api key: ")
  LASTFM_USER_NAME = input("username: ")

  if LASTFM_USER_NAME is None or LASTFM_API_KEY is None:
      print(
          """
          You need to generate some credentials, see the source code
          """
      )
      sys.exit(1)


  def get_scrobbles(
      endpoint="recenttracks", #possibly change to getTopArtists
      username = LASTFM_USER_NAME,
      key = LASTFM_API_KEY,
      limit = 200,
      extended = 0,
      page = 1,
      pages = 0,
  ):
      """
      endpoint: API endpoint.
      username: Last.fm username to fetch scrobbles for.
      key: API key.
      limit: The number of records per page. Maximum is 200.
      extended: Extended results from API, such as whether user has "liked" the track.
      page: First page to retrieve.
      pages: How many pages of results after "page" argument to retrieve. If 0, get all pages.
      """
      # initialize URL and lists to contain response fields
      url = (
          "https://ws.audioscrobbler.com/2.0/?method=user.get{}"
          "&user={}"
          "&api_key={}"
          "&limit={}"
          "&extended={}"
          "&page={}"
          "&format=json"
      )

      # get total number of pages
      request_url = url.format(endpoint, username, key, limit, extended, page)
      response = requests.get(request_url).json()
      total_pages = int(response[endpoint]["@attr"]["totalPages"])
      if pages > 0:
          total_pages = min([total_pages, pages])

      print(
          "Total pages to retrieve: {}. Estimated time: {}".format(
              total_pages, get_time_remaining(total_pages)
          )
      )

      artist_names = []
      album_names = []
      track_names = []
      timestamps = []
      urls = []
      # add formatted URLs to list to be requested in thread pool
      for page in range(0, int(total_pages) + 1, 1):
          urls.append(url.format(endpoint, username, key, limit, extended, page))
      p = pool.Pool.from_urls(urls)
      p.join_all()

      for response in p.responses():
          if endpoint in response.json():
              response_json = response.json()[endpoint]["track"]
              for track in response_json:
                  if "@attr" not in track:
                      artist_names.append(track["artist"][TEXT])
                      album_names.append(track["album"][TEXT])
                      track_names.append(track["name"])
                      timestamps.append(track["date"]["uts"])

      # create and populate a dataframe to contain the data
      df = pd.DataFrame()
      df["artist"] = artist_names
      df["album"] = album_names
      df["track"] = track_names
      df["timestamps"] = timestamps

      # In UTC. Last.fm returns datetimes in the user's locale when they listened
      df["datetime"] = pd.to_datetime(timestamps, unit="s")

      df["username"] = username
      
      df.sort_values("timestamps", ascending=False, inplace=True)
      return df

  def get_time_remaining(pages_remaining):
      """Calculate the estimated time remaining."""
      millis_remaining = int(
          (pages_remaining * ESTIMATED_TIME_FOR_PROCESSING_PAGE)
          + (
              pages_remaining
              * ESTIMATED_TIME_FOR_PROCESSING_DATAFRAME_PER_PAGE_OF_RESULTS
          )
      )
      seconds_remaining = (millis_remaining / 1000) % 60
      seconds_remaining = int(seconds_remaining)
      minutes_remaining = (millis_remaining / (1000 * 60)) % 60
      minutes_remaining = int(minutes_remaining)
      return "{}m{:2}s".format(minutes_remaining, seconds_remaining)

  scrobbles = get_scrobbles(page=2, pages=0)  # Default to all Scrobbles
  #scrobbles.to_csv("./data/lastfm_scrobbles.csv", index=False, encoding="utf-8")
  print("{:,} total rows".format(len(scrobbles)))

  list1 = scrobbles[["artist","username"]]

  list1["playcount"] = 1
  list1 = list1.groupby(["artist","username"]).count()
  list1 = list1.sort_values("playcount",ascending=False)
  list1 = list1.reset_index()

  group_df = pd.DataFrame(columns = ["artist",	"username",	"playcount"])
  group_df = group_df.append(list1)
  

group_df

how many friends are in your group?: 1
api key: 912b321d8a267e7892932b6deccb3d2c
username: metaversegods
Total pages to retrieve: 1. Estimated time: 0m 0s
89 total rows


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,artist,username,playcount
0,Theevs,metaversegods,11
1,Satori,metaversegods,4
2,Crépon,metaversegods,3
3,Rome Hero Foxes,metaversegods,2
4,Christian Nodal,metaversegods,2
...,...,...,...
60,Hermanos Gutiérrez,metaversegods,1
61,Intocable,metaversegods,1
62,Jaeden Camstra,metaversegods,1
63,JinSang,metaversegods,1


In [ ]:
artist = input("Hello artist! Your name is: ")
fans = group_df[group_df["artist"]==artist]
fans

Hello artist! Your name is: Theevs


,artist,username,playcount
0,Theevs,metaversegods,11
